In [1]:
import tensorflow as tf
from tensorflow.contrib import rnn
import os
import os.path

class main(object):
    
    PATH_DATASET = '/media/ikesan009/B418B4D718B499B6/research/CENSREC/dataset'
    MFCC_DIM = 39
    BATCH_SIZE = 30
    LEN_SEQ = 300
    N_CLASS = 22
    
    #TFRecordsからデータセット取り出し
    def input(self, path_tfr, MFCC_DIM, BATCH_SIZE):
        file_name_queue = tf.train.string_input_producer([path_tfr])
        reader = tf.TFRecordReader()
        _, serialized_example = reader.read(file_name_queue)
        features = tf.parse_single_example(
            serialized_example,
            features={
                'label': tf.FixedLenFeature([], tf.int64),
                'data': tf.FixedLenFeature([], tf.string),
            })
        
        datas = tf.decode_raw(features['data'], tf.float32)
        labels = tf.cast(features['label'], tf.int32)
        
        datas = tf.reshape(datas, [self.LEN_SEQ, MFCC_DIM])
        labels = tf.reshape(labels, [1])

        datas, labels = tf.train.shuffle_batch(
            [datas, labels],
            batch_size=BATCH_SIZE,capacity=1000+MFCC_DIM*BATCH_SIZE,min_after_dequeue=1000
        )
            
        return datas, labels

    #RNN
    def RNN(self, x):
        x = tf.unstack(x, self.LEN_SEQ, 1)

        # LSTMセルを定義する
        lstm_cell = rnn.BasicLSTMCell(256, forget_bias=1.0)

        # モデルの定義。各タイムステップの出力値と状態が返される
        outputs, states = rnn.static_rnn(lstm_cell, x, dtype=tf.float32)

        # 変数定義
        weight = tf.Variable(tf.random_normal([256, self.N_CLASS]))
        bias = tf.Variable(tf.random_normal([self.N_CLASS]))

        return tf.matmul(outputs[-1], weight) + bias
    
    #トレーニング部
    def train(self):
        x = tf.placeholder(tf.float32, shape=[None, self.LEN_SEQ, self.MFCC_DIM])
        y_ = tf.placeholder(tf.float32, shape=[None, self.N_CLASS])

        preds = self.RNN(x)

        cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=preds, labels=y_))
        optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)
        
        correct_pred = tf.equal(tf.argmax(preds, 1), tf.argmax(y_, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))
        
        data, label = self.input(os.path.join(self.PATH_DATASET, 'train-male.tfrecords'), self.MFCC_DIM, self.BATCH_SIZE)
        label = tf.one_hot(label, depth = self.N_CLASS, dtype = tf.float32)
        
        n_training_iters = 100000 * self.BATCH_SIZE
        init_op = [tf.global_variables_initializer(), tf.local_variables_initializer()]
        
        with tf.Session() as sess:
            sess.run(init_op)
            coord = tf.train.Coordinator()
            threads = tf.train.start_queue_runners(sess=sess, coord=coord)
            step = 1
            while step * self.BATCH_SIZE < n_training_iters:
                batch = sess.run([data, label])
                batch[1] = batch[1].reshape([-1, self.N_CLASS])
                sess.run(optimizer, feed_dict={x: batch[0], y_:batch[1]})
                if step % 10 == 0:
                    acc = sess.run(accuracy, feed_dict={x: batch[0], y_: batch[1]})
                    loss = sess.run(cost, feed_dict={x: batch[0], y_: batch[1]})
                    print('step: {} / loss: {:.6f} / acc: {:.5f}'.format(step, loss, acc))
                step += 1

            """
            # テスト
            test_len = 128
            test_batch = self.input(os.path.join(self.PATH_DATASET, 'test-male.tfrecords'), self.MFCC_DIM, test_len)
            test_data, test_label = sess.run(test_batch)
            test_acc = sess.run(accuracy, feed_dict={x: test_data, y: test_label})
            print("Test Accuracy: {}".format(test_acc))
            """
            coord.request_stop()
            coord.join(threads)

In [ ]:
m = main()
#m.input(os.path.join(m.PATH_DATASET, 'train-male.tfrecords'), m.MFCC_DIM, m.BATCH_SIZE)
m.train()

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See @{tf.nn.softmax_cross_entropy_with_logits_v2}.

step: 10 / loss: 3.286863 / acc: 0.00000
step: 20 / loss: 3.223075 / acc: 0.03333
step: 30 / loss: 3.019622 / acc: 0.06667
step: 40 / loss: 3.189710 / acc: 0.00000
step: 50 / loss: 3.162052 / acc: 0.06667
step: 60 / loss: 3.216613 / acc: 0.03333
step: 70 / loss: 3.128908 / acc: 0.03333
step: 80 / loss: 3.132331 / acc: 0.10000
step: 90 / loss: 3.120088 / acc: 0.00000
step: 100 / loss: 3.110112 / acc: 0.00000
step: 110 / loss: 3.144465 / acc: 0.06667
step: 120 / loss: 3.154385 / acc: 0.06667
step: 130 / loss: 3.143729 / acc: 0.06667
step: 140 / loss: 3.071159 / acc: 0.06667
step: 150 / loss: 3.100992 / acc: 0.03333
step: 160 / loss: 3.147342 / acc: 0.06667
step: 170 / loss: 3.092034 / acc: 0.03333
step: 180 / loss: 3.143621 / acc: 0.03333
step: 190 / loss: 3.069518 / acc: 0.03333
step: 200 / loss: 3

step: 1860 / loss: 3.099900 / acc: 0.00000
step: 1870 / loss: 3.087710 / acc: 0.00000
step: 1880 / loss: 3.106209 / acc: 0.03333
step: 1890 / loss: 3.066225 / acc: 0.03333
step: 1900 / loss: 3.113682 / acc: 0.00000
step: 1910 / loss: 3.081240 / acc: 0.03333
step: 1920 / loss: 3.152207 / acc: 0.00000
step: 1930 / loss: 3.097428 / acc: 0.03333
step: 1940 / loss: 3.131402 / acc: 0.03333
step: 1950 / loss: 3.088255 / acc: 0.00000
step: 1960 / loss: 3.067945 / acc: 0.06667
step: 1970 / loss: 3.096052 / acc: 0.03333
step: 1980 / loss: 3.060540 / acc: 0.10000
step: 1990 / loss: 3.045642 / acc: 0.06667
step: 2000 / loss: 3.096565 / acc: 0.03333
step: 2010 / loss: 3.084502 / acc: 0.03333
step: 2020 / loss: 3.117146 / acc: 0.10000
step: 2030 / loss: 3.107150 / acc: 0.03333
step: 2040 / loss: 3.057824 / acc: 0.03333
step: 2050 / loss: 3.109582 / acc: 0.03333
step: 2060 / loss: 3.097394 / acc: 0.03333
step: 2070 / loss: 3.105355 / acc: 0.03333
step: 2080 / loss: 3.086555 / acc: 0.00000
step: 2090 

step: 3770 / loss: 3.109689 / acc: 0.00000
step: 3780 / loss: 3.096031 / acc: 0.06667
step: 3790 / loss: 3.112941 / acc: 0.06667
step: 3800 / loss: 3.102005 / acc: 0.03333
step: 3810 / loss: 3.085505 / acc: 0.00000
step: 3820 / loss: 3.082733 / acc: 0.06667
step: 3830 / loss: 3.092904 / acc: 0.03333
step: 3840 / loss: 3.078431 / acc: 0.10000
step: 3850 / loss: 3.099363 / acc: 0.03333
step: 3860 / loss: 3.079284 / acc: 0.06667
step: 3870 / loss: 3.119755 / acc: 0.03333
step: 3880 / loss: 3.105454 / acc: 0.10000
step: 3890 / loss: 3.106450 / acc: 0.06667
step: 3900 / loss: 3.072355 / acc: 0.03333
step: 3910 / loss: 3.096549 / acc: 0.03333
step: 3920 / loss: 3.113513 / acc: 0.10000
step: 3930 / loss: 3.087064 / acc: 0.03333
step: 3940 / loss: 3.091971 / acc: 0.03333
step: 3950 / loss: 3.135488 / acc: 0.00000
step: 3960 / loss: 3.107649 / acc: 0.03333
step: 3970 / loss: 3.118599 / acc: 0.06667
step: 3980 / loss: 3.070377 / acc: 0.06667
step: 3990 / loss: 3.095540 / acc: 0.06667
step: 4000 

step: 5680 / loss: 3.096476 / acc: 0.00000
step: 5690 / loss: 3.128875 / acc: 0.00000
step: 5700 / loss: 3.074969 / acc: 0.06667
step: 5710 / loss: 3.121425 / acc: 0.03333
step: 5720 / loss: 3.081519 / acc: 0.06667
step: 5730 / loss: 3.093925 / acc: 0.06667
step: 5740 / loss: 3.108444 / acc: 0.06667
step: 5750 / loss: 3.084746 / acc: 0.03333
step: 5760 / loss: 3.117469 / acc: 0.00000
step: 5770 / loss: 3.095059 / acc: 0.10000
step: 5780 / loss: 3.111808 / acc: 0.03333
step: 5790 / loss: 3.070290 / acc: 0.03333
step: 5800 / loss: 3.114095 / acc: 0.03333
step: 5810 / loss: 3.113350 / acc: 0.03333
step: 5820 / loss: 3.107429 / acc: 0.00000
step: 5830 / loss: 3.052614 / acc: 0.10000
step: 5840 / loss: 3.103553 / acc: 0.03333
step: 5850 / loss: 3.110769 / acc: 0.00000
step: 5860 / loss: 3.100787 / acc: 0.03333
step: 5870 / loss: 3.086168 / acc: 0.00000
step: 5880 / loss: 3.112515 / acc: 0.03333
step: 5890 / loss: 3.113173 / acc: 0.00000
step: 5900 / loss: 3.066643 / acc: 0.03333
step: 5910 

step: 7590 / loss: 3.092086 / acc: 0.06667
step: 7600 / loss: 3.092795 / acc: 0.10000
step: 7610 / loss: 3.093738 / acc: 0.10000
step: 7620 / loss: 3.083038 / acc: 0.06667
step: 7630 / loss: 3.085395 / acc: 0.10000
step: 7640 / loss: 3.077412 / acc: 0.06667
step: 7650 / loss: 3.087431 / acc: 0.13333
step: 7660 / loss: 3.064246 / acc: 0.03333
step: 7670 / loss: 3.088764 / acc: 0.03333
step: 7680 / loss: 3.098687 / acc: 0.10000
step: 7690 / loss: 3.078195 / acc: 0.06667
step: 7700 / loss: 3.105163 / acc: 0.00000
step: 7710 / loss: 3.075496 / acc: 0.06667
step: 7720 / loss: 3.075969 / acc: 0.10000
step: 7730 / loss: 3.106746 / acc: 0.03333
step: 7740 / loss: 3.115789 / acc: 0.03333
step: 7750 / loss: 3.101948 / acc: 0.00000
step: 7760 / loss: 3.086252 / acc: 0.03333
step: 7770 / loss: 3.100387 / acc: 0.00000
step: 7780 / loss: 3.091138 / acc: 0.03333
step: 7790 / loss: 3.087053 / acc: 0.06667
step: 7800 / loss: 3.106921 / acc: 0.00000
step: 7810 / loss: 3.108994 / acc: 0.00000
step: 7820 

...

step: 16990 / loss: 3.102360 / acc: 0.03333
step: 17000 / loss: 3.086747 / acc: 0.10000
step: 17010 / loss: 3.083473 / acc: 0.00000
step: 17020 / loss: 3.108591 / acc: 0.10000
step: 17030 / loss: 3.085856 / acc: 0.13333
step: 17040 / loss: 3.081771 / acc: 0.10000
step: 17050 / loss: 3.089461 / acc: 0.13333
step: 17060 / loss: 3.079721 / acc: 0.10000
step: 17070 / loss: 3.094408 / acc: 0.03333
step: 17080 / loss: 3.086109 / acc: 0.00000
step: 17090 / loss: 3.089172 / acc: 0.06667
step: 17100 / loss: 3.090713 / acc: 0.06667
step: 17110 / loss: 3.092295 / acc: 0.06667
step: 17120 / loss: 3.088406 / acc: 0.03333
step: 17130 / loss: 3.099154 / acc: 0.03333
step: 17140 / loss: 3.080337 / acc: 0.03333
step: 17150 / loss: 3.094057 / acc: 0.00000
step: 17160 / loss: 3.097020 / acc: 0.06667
step: 17170 / loss: 3.090298 / acc: 0.00000
step: 17180 / loss: 3.105983 / acc: 0.00000
step: 17190 / loss: 3.090905 / acc: 0.00000
step: 17200 / loss: 3.088511 / acc: 0.10000
step: 17210 / loss: 3.102051 / a

step: 18860 / loss: 3.095074 / acc: 0.03333
step: 18870 / loss: 3.098558 / acc: 0.00000
step: 18880 / loss: 3.086182 / acc: 0.06667
step: 18890 / loss: 3.092979 / acc: 0.00000
step: 18900 / loss: 3.092567 / acc: 0.03333
step: 18910 / loss: 3.100576 / acc: 0.00000
step: 18920 / loss: 3.086110 / acc: 0.03333
step: 18930 / loss: 3.107366 / acc: 0.00000
step: 18940 / loss: 3.091305 / acc: 0.06667
step: 18950 / loss: 3.111449 / acc: 0.00000
step: 18960 / loss: 3.096222 / acc: 0.03333
step: 18970 / loss: 3.083896 / acc: 0.06667
step: 18980 / loss: 3.088231 / acc: 0.03333
step: 18990 / loss: 3.092388 / acc: 0.00000
step: 19000 / loss: 3.087056 / acc: 0.13333
step: 19010 / loss: 3.103899 / acc: 0.00000
step: 19020 / loss: 3.097135 / acc: 0.03333
step: 19030 / loss: 3.089196 / acc: 0.03333
step: 19040 / loss: 3.088832 / acc: 0.06667
step: 19050 / loss: 3.095633 / acc: 0.06667
step: 19060 / loss: 3.091557 / acc: 0.00000
step: 19070 / loss: 3.099100 / acc: 0.13333
step: 19080 / loss: 3.082595 / a

step: 20730 / loss: 3.096967 / acc: 0.03333
step: 20740 / loss: 3.096504 / acc: 0.03333
step: 20750 / loss: 3.103705 / acc: 0.00000
step: 20760 / loss: 3.092510 / acc: 0.06667
step: 20770 / loss: 3.091791 / acc: 0.06667
step: 20780 / loss: 3.090487 / acc: 0.06667
step: 20790 / loss: 3.095587 / acc: 0.00000
step: 20800 / loss: 3.087951 / acc: 0.03333
step: 20810 / loss: 3.093127 / acc: 0.10000
step: 20820 / loss: 3.106276 / acc: 0.00000
step: 20830 / loss: 3.075327 / acc: 0.10000
step: 20840 / loss: 3.080910 / acc: 0.10000
step: 20850 / loss: 3.083005 / acc: 0.03333
step: 20860 / loss: 3.092277 / acc: 0.00000
step: 20870 / loss: 3.096391 / acc: 0.03333
step: 20880 / loss: 3.096567 / acc: 0.03333
step: 20890 / loss: 3.096254 / acc: 0.06667
step: 20900 / loss: 3.102368 / acc: 0.00000
step: 20910 / loss: 3.092458 / acc: 0.03333
step: 20920 / loss: 3.087882 / acc: 0.13333
step: 20930 / loss: 3.089479 / acc: 0.03333
step: 20940 / loss: 3.100032 / acc: 0.00000
step: 20950 / loss: 3.103028 / a

step: 22600 / loss: 3.093820 / acc: 0.06667
step: 22610 / loss: 3.089662 / acc: 0.00000
step: 22620 / loss: 3.083999 / acc: 0.06667
step: 22630 / loss: 3.093971 / acc: 0.03333
step: 22640 / loss: 3.094699 / acc: 0.06667
step: 22650 / loss: 3.091839 / acc: 0.03333
step: 22660 / loss: 3.101394 / acc: 0.00000
step: 22670 / loss: 3.099607 / acc: 0.00000
step: 22680 / loss: 3.089421 / acc: 0.06667
step: 22690 / loss: 3.087256 / acc: 0.06667
step: 22700 / loss: 3.087459 / acc: 0.06667
step: 22710 / loss: 3.097270 / acc: 0.03333
step: 22720 / loss: 3.098293 / acc: 0.06667
step: 22730 / loss: 3.096084 / acc: 0.03333
step: 22740 / loss: 3.092781 / acc: 0.00000
step: 22750 / loss: 3.098173 / acc: 0.03333
step: 22760 / loss: 3.083798 / acc: 0.03333
step: 22770 / loss: 3.090818 / acc: 0.00000
step: 22780 / loss: 3.107553 / acc: 0.06667
step: 22790 / loss: 3.081717 / acc: 0.06667
step: 22800 / loss: 3.086507 / acc: 0.06667
step: 22810 / loss: 3.096800 / acc: 0.00000
step: 22820 / loss: 3.083527 / a

step: 24470 / loss: 3.102418 / acc: 0.03333
step: 24480 / loss: 3.095439 / acc: 0.06667
step: 24490 / loss: 3.096694 / acc: 0.00000
step: 24500 / loss: 3.088891 / acc: 0.13333
step: 24510 / loss: 3.090844 / acc: 0.03333
step: 24520 / loss: 3.100078 / acc: 0.03333
step: 24530 / loss: 3.093385 / acc: 0.06667
step: 24540 / loss: 3.095889 / acc: 0.06667
step: 24550 / loss: 3.111536 / acc: 0.00000
step: 24560 / loss: 3.080969 / acc: 0.10000
step: 24570 / loss: 3.084411 / acc: 0.03333
step: 24580 / loss: 3.096156 / acc: 0.06667
step: 24590 / loss: 3.090457 / acc: 0.03333
step: 24600 / loss: 3.088220 / acc: 0.00000
step: 24610 / loss: 3.102199 / acc: 0.00000
step: 24620 / loss: 3.091716 / acc: 0.06667
step: 24630 / loss: 3.074667 / acc: 0.03333
step: 24640 / loss: 3.091465 / acc: 0.03333
step: 24650 / loss: 3.114115 / acc: 0.00000
step: 24660 / loss: 3.082982 / acc: 0.10000
step: 24670 / loss: 3.090029 / acc: 0.06667
step: 24680 / loss: 3.091123 / acc: 0.00000
step: 24690 / loss: 3.079391 / a